# Example Scenario

This notebook contains an example setup to run experiments as done in the [paper](https://arxiv.org/abs/2004.07641).

As we are in the process of significantly refactoring the code base and experiments, we did not update the entire experiment notebook done to reproduce the paper experiments, as they are now deprecated.

__If you want to play around and reproduce results currently shown in the arXiv paper, revert to commit__
`28b14a1dca53e12573eabf99317b2c7517c81761`

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

**Import libs**

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
import numpy as np
import random as rd
import pandas as pd
import pickle
import multiprocessing
import argparse
from lib.measures import *
from lib.experiment import Experiment, options_to_str, process_command_line
from lib.calibrationSettings import calibration_lockdown_dates, calibration_mob_paths, calibration_states, calibration_lockdown_beta_multipliers
from lib.calibrationFunctions import get_calibrated_params

import matplotlib.pyplot as plt
from lib.plot import Plotter
%matplotlib inline

# converting days to hours
TO_HOURS = 24.0

## 1. Define the experiment parameters

In [ ]:
name = 'example-experiment'

country = 'GER'
area = 'TU'
start_date = '2020-07-01'
end_date = '2020-07-31'

random_repeats = 6 # recommended to set to at least 48  
full_scale = False

# seed
c = 0
np.random.seed(c)
rd.seed(c)

The model parameters should be calibrated using Bayesian optimization. 
Run `python calibrate.py --help` to see how to use the calibration script. Also, refer to `calibrationSettings.py` for detailed settings and automatic loading of results.

In [ ]:
arbitrary_model_params = {
    'beta_site' :  0.8,      # risk of exposure at sites
    'beta_household' : 0.8,  # risk of exposure at home
    'p_stay_home' : 0.5,     # probability of not attending a given visit to a site due to social distancing
}

#### Define Experiment object

In [ ]:
experiment_info = f'{name}-{country}-{area}'
experiment = Experiment(
    experiment_info=experiment_info,
    start_date=start_date,
    end_date=end_date,
    random_repeats=random_repeats,
    full_scale=full_scale,
    verbose=True,
)

#### Define Measures to take place during simulation.
See `measures.py` for other options.

In [ ]:
lockdown_at_day = 14

sim_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
max_time = TO_HOURS * sim_days # in hours

example_measures = [
    
    # education, social sites, and offices partially closed/reduced infection after 1 week
    BetaMultiplierMeasureByType(
        t_window=Interval(lockdown_at_day * TO_HOURS, max_time), 
        beta_multiplier={ 
            'education': 0.5, 
            'social': 0.5, 
            'bus_stop': 1.0, 
            'office': 0.5, 
            'supermarket': 1.0
        }),
    
    # less activities of all due to contact constraints after 14 days
    SocialDistancingForAllMeasure(
     t_window=Interval(lockdown_at_day * TO_HOURS, max_time), 
        p_stay_home=arbitrary_model_params['p_stay_home'])
]


#### Add Simulation to experiment

In [ ]:
simulation_info = options_to_str(
    descr='example-simulation',
)

experiment.add(
    simulation_info=simulation_info,
    country=country,
    area=area,
    measure_list=example_measures,
    set_calibrated_params_to=arbitrary_model_params,
    full_scale=full_scale,
    test_update=None,
    seed_summary_path=None,
    set_initial_seeds_to=None)

print(f'{experiment_info} configuration done.')

## 2. Run the simulations

Run the experiment using the parameters defined above and simulate in the future, with additional measures of varying duration.

**WARNING: this cell might take a long time to run depending of the parameters defined above**

In [ ]:
experiment.run_all()


## 3. Plot the results

Refer to `sim-plot.ipynb` for plotting. The results of the above experiment are stored inside the `sim/summaries/` folder of the repository.